# FRD Accomplishments Script
# Read the README File in Github for more detailed information.  https://github.com/jgorman-tfs/FRD-Accomplishments <br> 
# Data Source: 

## Set you folder paths and qtr name here

In [ ]:
#Part 1 Initialization

#Quarter and fiscal year
qtr = "Q4_2024"
#Set folder path to new quarter folder
folder_path = r'D:\ArcGIS_Projects\FRDAccomplishments\FY2024Q4'
#Set the geodatabase path
gdb = r'D:\ArcGIS_Projects\FRDAccomplishments\FRDAccomplishments.gdb'
#This shapefile is provided in sharepoint. If your using your own, you MUST calculate a new field with the County names of each city.
cities = "Texas_Places_WithCounties"
#Path to the excel file provided by Mac
excel_path = r"D:\ArcGIS_Projects\FRDAccomplishments\FY2024Q4\MAM_UCF_Copy.xls"
#Sheet name of said excel file that has the edited data
sheet_name = "CountyFinder2"
#This is a temperory csv, it doesn't really matter where it goes
output_path = os.path.join(r'D:\ArcGIS_Projects\FRDAccomplishments', 'temp.csv')
#Name of yout county layer in the geodatabase
counties_template = "Template"

#Part 2 Initialization
#Path to excel file recieved from Brad
excel_path_Brad = r'D:\ArcGIS_Projects\FRDAccomplishments\FY2024Q4\CountyAccomplishments_FromBrad_FY24Q3_Original.csv'
#Sheet names of edited data from Mac
sheet_name_ELMR = "CountyFinder1"
sheet_name_SPAM = "CountyFinder2"

ELMR_accomp_temp = os.path.join(folder_path, 'ELMR_accomp_temp.csv')
SPAM_accomp_temp = os.path.join(folder_path, 'SPAM_accomp_temp.csv')

## Part 1: Find the Acres for Urban and Forestry Communities Served

Change the name of the qtr variable to the current quarter and fiscal year and change the path to the right excel file. Ensure the other variables are pointing to the correct folders. It is best to use the Texas_Places_WithCounties from my sharepoint. If you want to create your own, **make sure you add a field called "Testing" and set the values equal to 1.**

In [3]:
# Import necessary packages
import pandas as pd

# Create a new map with the name as the variable
aprx = arcpy.mp.ArcGISProject("CURRENT")
new_map = aprx.createMap(qtr, "MAP")
aprx.save()

# Import a table from an Excel file

df = pd.read_excel(excel_path, sheet_name=sheet_name)


old_column_list = df.columns.tolist()

print("old column headers:", old_column_list)
new_column_names = {
    old_column_list[0]: "CityName",
    old_column_list[1]: "AssistType",
    old_column_list[2]: "Category"
}
df.rename(columns=new_column_names, inplace=True)
print(df.head(5))

old column headers: ['PlaceName', 'AssistTypeName', 'Category']
     CityName                     AssistType                Category
0      Austin  Arbor Day/Tree City USA Event  Conservation Education
1       Hutto  Arbor Day/Tree City USA Event  Conservation Education
2  San Marcos  Arbor Day/Tree City USA Event  Conservation Education
3      Hudson  Arbor Day/Tree City USA Event  Conservation Education
4  Huntsville  Arbor Day/Tree City USA Event  Conservation Education


Import the table into your gdb

In [3]:
import os

# Convert the DataFrame to an Excel file
df.to_csv(output_path, index=False)

print(f"Excel file saved to: {output_path}")
arcpy.TableToTable_conversion(output_path, gdb, f"{qtr}_Cities")

Excel file saved to: D:\ArcGIS_Projects\FRDAccomplishments\temp.csv


<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Cities'>

Join fields from Texas_Places_WithCounties into your newly imported table. This will let you figure out which cities need to be fixed.

In [4]:
# Join field from copied feature to the imported table
in_data = f"{gdb}\\{qtr}_Cities"
in_field = "CityName"
join_table = f"{gdb}\\{cities}"
join_field = "CityName"
fields = "Testing"
arcpy.management.JoinField(in_data, in_field, join_table, join_field, fields)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Cities'>

### After running the above code, you will likely have some null values in the Testing column. This is because the cities aren't matching up with the Texas_Places_WithCounties for various reasons. Figure out why and edit the names so they match up with your imported table. Delete the Testing column in the imported table, and re run the above section of code until you do not have any null values. 

Create a new field in the table that will be used to calculate acres

In [6]:
in_table = f"{gdb}\\{qtr}_Cities"
field_name = f"{qtr}_Acres"
field_type = "FLOAT"
field_precision = 2
arcpy.management.AddField(in_table, field_name, field_type, field_precision)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Cities'>

Set field to 0

In [7]:
expression = 0
arcpy.management.CalculateField(in_table, field_name, expression)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Cities'>

Join fields from table back to city layer

In [8]:
texas_cities = f"{gdb}\\{cities}"
in_field = "CityName"
imported_table = f"{gdb}\\{qtr}_Cities"
join_field = "CityName"
fields = field_name
arcpy.management.JoinField(texas_cities, in_field, imported_table, join_field, fields)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Texas_Places_WithCounties'>

The code block below will select features in both the city layer and table to see if the number of cities agree with each other. If they do not agree, you will need to figure out why. 

In [11]:
arcpy.management.SelectLayerByAttribute(imported_table, "NEW_SELECTION", f"{fields} = 0")
num_cities_intable = int(arcpy.management.GetCount(imported_table)[0])
print(f"Number of cities in the Imported Table: {num_cities_intable}")
arcpy.management.SelectLayerByAttribute(imported_table, "CLEAR_SELECTION")

arcpy.management.SelectLayerByAttribute(cities, "NEW_SELECTION", f"{field_name} = 0")
num_cities_incities = int(arcpy.management.GetCount(cities)[0])
print(f"Number of cities in the Texas_Places_WithCounties: {num_cities_incities}")
arcpy.management.SelectLayerByAttribute(cities, "CLEAR_SELECTION")

if num_cities_intable != num_cities_incities:
    print("There is something wrong. The number of cities in the imported table and the feature layer do not match.")
else:
    print("Continue onto the next step")

Number of cities in the Imported Table: 54
Number of cities in the Texas_Places_WithCounties: 31
There is something wrong. The number of cities in the imported table and the feature layer do not match.


The code block below calculates the area of the selected cities. 

In [13]:
arcpy.management.SelectLayerByAttribute(cities, "NEW_SELECTION", f"{field_name} = 0")
num_cities_incities = int(arcpy.management.GetCount(cities)[0])
print(f"Number of cities in the Texas_Places_WithCounties: {num_cities_incities}")


length_unit = "MILES_US"
area_unit = "ACRES"

# Define the WKID (Well-Known ID) of the coordinate system you want to use
wkid = 3665  # Example: WGS 1984 Geographic Coordinate System

try:
    # Create a spatial reference object using the specified WKID
    spatial_ref = arcpy.SpatialReference(3665)

    # Calculate geometry attributes with specified coordinate system and units
    arcpy.management.CalculateGeometryAttributes(cities,
                                                 [[field_name, "AREA_GEODESIC"]],
                                                 length_unit, area_unit,
                                                 spatial_ref)

    print("Geometry attributes calculated successfully.")

except arcpy.ExecuteError as e:
    print(f"ExecuteError: {e}")
except Exception as e:
    print(f"Error occurred: {e}")

Number of cities in the Texas_Places_WithCounties: 31
Geometry attributes calculated successfully.


Run Summary Statistics to get the total acerage.

In [16]:
community_acres = f"{gdb}\\{qtr}_CommunityAcres"
arcpy.analysis.Statistics(cities, community_acres, [[field_name, "SUM"]])

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_CommunityAcres'>

## Part 2: Get the Accomplishments by County

In [41]:
#Create a copy of the County Layer that will be used for the map
counties_new = f"{qtr}_Counties"
arcpy.FeatureClassToFeatureClass_conversion(f"{gdb}\\{counties_template}", gdb, counties_new)

In [2]:
import pandas as pd
from openpyxl import load_workbook

accomp_from_ELMR_df = pd.read_excel(excel_path, sheet_name=sheet_name_ELMR)

accomp_from_SPAM_df = pd.read_excel(excel_path, sheet_name=sheet_name_SPAM)


old_column_list_ELMR = accomp_from_ELMR_df.columns.tolist()
old_column_list_SPAM = accomp_from_SPAM_df.columns.tolist()

new_column_names_ELMR = {
    old_column_list_ELMR[0]: "CountyName",
    old_column_list_ELMR[1]: "AssistType",
    old_column_list_ELMR[2]: "Category"
}
accomp_from_ELMR_df.rename(columns=new_column_names_ELMR, inplace=True)
print(accomp_from_ELMR_df.head(6))

new_column_names_SPAM = {
    old_column_list_SPAM[0]: "CityName",
    old_column_list_SPAM[1]: "AssistType",
    old_column_list_SPAM[2]: "Category"
}
accomp_from_SPAM_df.rename(columns=new_column_names_SPAM, inplace=True)
print(accomp_from_SPAM_df.head(6))

  CountyName            AssistType                Category
0   Anderson   Tree Planting Event    Technical Assistance
1   Angelina  UF Individual Assist    Technical Assistance
2      Bowie     Arbor Day Program  Conservation Education
3      Bowie     Arbor Day Program  Conservation Education
4       Cass   Tree Planting Event    Technical Assistance
5      Gregg  UF Individual Assist    Technical Assistance
     CityName                     AssistType                Category
0      Austin  Arbor Day/Tree City USA Event  Conservation Education
1       Hutto  Arbor Day/Tree City USA Event  Conservation Education
2  San Marcos  Arbor Day/Tree City USA Event  Conservation Education
3      Hudson  Arbor Day/Tree City USA Event  Conservation Education
4  Huntsville  Arbor Day/Tree City USA Event  Conservation Education
5      Lufkin  Arbor Day/Tree City USA Event  Conservation Education


In [3]:
import os
ELMR_accomp_temp = os.path.join(folder_path, 'ELMR_accomp_temp.csv')
SPAM_accomp_temp = os.path.join(folder_path, 'SPAM_accomp_temp.csv')
# Convert the DataFrame to an Excel file
accomp_from_ELMR_df.to_csv(ELMR_accomp_temp, index=False)
accomp_from_SPAM_df.to_csv(SPAM_accomp_temp, index=False)

arcpy.TableToTable_conversion(ELMR_accomp_temp, gdb, f"{qtr}_Counties_FromELMR")
arcpy.TableToTable_conversion(SPAM_accomp_temp, gdb, f"{qtr}_Cities_FromSPAM")
arcpy.TableToTable_conversion(accomp_from_Brad, gdb, f"{qtr}_Counties_FromBrad")

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Counties_FromBrad'>

In [6]:
in_data = f"{gdb}\\{qtr}_Cities_FromSPAM"
in_field = "CityName"
join_table = f"{gdb}\\{cities}"
join_field = "CityName"
fields = "CountyName"
arcpy.management.JoinField(in_data, in_field, join_table, join_field, fields)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Cities_FromSPAM'>

In [8]:
inputs = [f"{gdb}\\{qtr}_Cities_FromSPAM", f"{gdb}\\{qtr}_Counties_FromELMR" ]
output = f"{gdb}\\{qtr}_Counties_FromUCF_Merge"
arcpy.management.Merge(inputs, output)

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Counties_FromUCF_Merge'>

In [9]:
output_table = f"{gdb}\\{qtr}_Counties_FromUCF"
arcpy.analysis.Statistics(output, output_table, [["CountyName", "COUNT"]], ["Category", "CountyName"])

<Result 'D:\\ArcGIS_Projects\\FRDAccomplishments\\FRDAccomplishments.gdb\\Q4_2024_Counties_FromUCF'>

In [8]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "NEW_SELECTION", "Conservation_Education IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Conservation_Education", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "NEW_SELECTION", "Management_Plans IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Management_Plans", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "NEW_SELECTION", "Technical_Assistance IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Technical_Assistance", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "NEW_SELECTION", "FIA_Plots IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "FIA_Plots", 0 )

arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "CLEAR_SELECTION")


<Result 'Q4_2024_Counties_FromBrad'>

In [16]:
arcpy.management.AddFields(f"{qtr}_Counties_FromBrad", ["CountyChecker", "SHORT"])
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "CountyChecker", 0 )
arcpy.management.JoinField(f"{qtr}_Counties_FromUCF", "CountyName", f"{qtr}_Counties_FromBrad" , "CountyName", "CountyChecker")



<Result 'Q4_2024_Counties_FromUCF'>

In [24]:
null_count = 0
with arcpy.da.SearchCursor(f"{qtr}_Counties_FromUCF", ["CountyChecker"]) as cursor:
    for row in cursor:
            # Check if the field value is None (NULL in ArcGIS)
        if row[0] is None:
            null_count += 1
    
if null_count == 0:
    print("All counties match, proceed to next step")
else:
    print(f"WAIT! There are {null_count} counties that are not matched in the UCF table. Go back and fix before proceeding.")

All counties match, proceed to next step


In [28]:
arcpy.management.AddFields(f"{qtr}_Counties_FromUCF", ["Conservation_Education_UCF", "Technical_Assistance_UCF"])

<Result 'Q4_2024_Counties_FromUCF'>

In [31]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "NEW_SELECTION", "Category = 'Conservation Education'")
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Conservation_Education_UCF", 0 )
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Conservation_Education_UCF", "!COUNT_CountyName!" )
arcpy.management.JoinField(f"{qtr}_Counties_FromBrad", "CountyName", f"{qtr}_Counties_FromUCF" , "CountyName", "Conservation_Education_UCF")

<Result 'Q4_2024_Counties_FromUCF'>

In [33]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "NEW_SELECTION", "Category = 'Technical Assistance'")
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", 0 )
arcpy.management.CalculateField(f"{qtr}_Counties_FromUCF", "Technical_Assistance_UCF", "!COUNT_CountyName!" )
arcpy.management.JoinField(f"{qtr}_Counties_FromBrad", "CountyName", f"{qtr}_Counties_FromUCF" , "CountyName", "Technical_Assistance_UCF")

<Result 'Q4_2024_Counties_FromBrad'>

In [34]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromUCF", "CLEAR_SELECTION")

<Result 'Q4_2024_Counties_FromUCF'>

In [38]:
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "NEW_SELECTION", "Technical_Assistance_UCF IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Technical_Assistance_UCF", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "NEW_SELECTION", "Conservation_Education_UCF IS NULL")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Conservation_Education_UCF", 0 )
arcpy.management.SelectLayerByAttribute(f"{qtr}_Counties_FromBrad", "CLEAR_SELECTION")

<Result 'Q4_2024_Counties_FromBrad'>

In [39]:
arcpy.management.AddFields(f"{qtr}_Counties_FromBrad", ["Conservation_Education_Total", "Technical_Assistance_Total"])
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Conservation_Education_Total", 0)
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Technical_Assistance_Total", 0)

<Result 'Q4_2024_Counties_FromBrad'>

In [40]:
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Conservation_Education_Total", "!Conservation_Education! + !Conservation_Education_UCF!")
arcpy.management.CalculateField(f"{qtr}_Counties_FromBrad", "Technical_Assistance_Total", "!Technical_Assistance! + !Technical_Assistance_UCF!")

<Result 'Q4_2024_Counties_FromBrad'>

In [45]:
arcpy.management.JoinField(counties_new, 
                           "CountyName", 
                           f"{qtr}_Counties_FromBrad" , 
                           "CountyName", 
                           "CountyChecker")

<Result 'Q4_2024_Counties'>

In [46]:
null_count = 0
with arcpy.da.SearchCursor(counties_new, ["CountyChecker"]) as cursor:
    for row in cursor:
            # Check if the field value is None (NULL in ArcGIS)
        if row[0] is None:
            null_count += 1
    
if null_count == 0:
    print("All counties match, proceed to next step")
else:
    print(f"WAIT! There are {null_count} counties that are not matched in the UCF table. Go back and fix before proceeding.")

WAIT! There are 1 counties that are not matched in the UCF table. Go back and fix before proceeding.


In [48]:
arcpy.management.JoinField(counties_new, 
                           "CountyName", 
                           f"{qtr}_Counties_FromBrad" , 
                           "CountyName", 
                           ["FIA_Plots", "Management_Plans", "Conservation_Education_Total", "Technical_Assistance_Total"])

<Result 'Q4_2024_Counties'>